In [18]:
import csv
from collections import Counter
import pandas as pd
from itertools import chain, combinations
from tabulate import tabulate
import time

In [19]:
# Read the .pb file 

path = "netherlands_amsterdam_166_.pb"  # set this variable
with open(path, 'r', newline='', encoding="utf-8") as csvfile:
    meta = {}
    projects = {}
    votes = {}
    section = ""
    header = []
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        if str(row[0]).strip().lower() in ["meta", "projects", "votes"]:
            section = str(row[0]).strip().lower()
            header = next(reader)
        elif section == "meta":
            meta[row[0]] = row[1].strip()
        elif section == "projects":
            projects[row[0]] = {}
            for it, key in enumerate(header[1:]):
                projects[row[0]][key.strip()] = row[it+1].strip()
        elif section == "votes":
            votes[row[0]] = {}
            for it, key in enumerate(header[1:]):
                votes[row[0]][key.strip()] = row[it+1].strip()

In [20]:
#Extract required data from the file
C=set(projects.keys())
V=set(votes.keys())
n=len(V)
approval_dict = {key: set(value['vote'].split(',')) for key, value in votes.items()}

In [21]:
lengths = [len(value) for value in approval_dict.values()]

# Calculate the average length
average_length = sum(lengths) / len(lengths)

print("Average length of values:", average_length)

Average length of values: 11.927230046948356


In [22]:
max_length = min(len(value) for value in approval_dict.values())
max_length

1

In [23]:
#Creating a df with the projects and the number of votes they got
all_values = set()
for value_set in approval_dict.values():
    all_values.update(value_set)

# Count occurrences of each value in the entire dictionary
count_dict = Counter()
for value_set in approval_dict.values():
    count_dict.update(value_set)

# Create a list with unique values and their counts
result_list = [[value, count_dict[value]] for value in all_values]

greedy_list = pd.DataFrame(result_list, columns=['Value', 'Count'])
greedy_list = greedy_list.sort_values(by='Count', ascending=False)

In [24]:
greedy_list.head(3)

,Value,Count
19,12437,242
33,12431,205
3,12422,167


In [25]:
# votes

In [32]:

# def powerset(iterable, max_size): 
#     s = list(iterable)
#     return chain.from_iterable(combinations(s, r) for r in range(1, min(len(s)+1, max_size+1)))
# def subset_of_len_l(C_modified, l):
#     return set(combinations(C_modified, l))

# # Define the function to create the set W dynamically based on its size
# def create_set_W(df, W):
#     return set(df.head(W)['Value'])
# # Define the function for JR
# def JR(non_winners, approval_dict, W_set, k, n):
#     start_time = time.time()  # Record start time
#     counts = {}
#     breaks_jr = False

#     for non_winner in non_winners:
#         counts[non_winner] = 0
#         for key, value in approval_dict.items():
#             if non_winner in value and not set(W_set).intersection(value):
#                 counts[non_winner] += 1
#                 if counts[non_winner] >= n / k:
#                     breaks_jr = True
#                     break
#         if breaks_jr:
#             return "Breaks JR."
        
#     end_time = time.time()  # Record end time
#     duration = end_time - start_time  # Calculate duration
#     return "Satisfies JR.", duration

# # Define the function for EJR
# def EJR(approval_lists, winners, k, n, C,l):
#     start_time = time.time()  # Record start time
    
#     #FOR L in 1 to k+1, 
#     #new C= where C have atleast nl/k votes 
#     C_modified = [candidate for candidate in C if greedy_list.loc[greedy_list['Value'] == candidate, 'Count'].iloc[0] >= l * n / k]
#     #print("lenght of C=",len(C))
#     #print("lenght of C_modified=",len(C_modified))
#     # Generate subsets S using C_modified
    
#     # S should be only len =l
#     S=subset_of_len_l(C_modified, l)
#     #S = set(s for s in powerset(C_modified, k) if not set(s).issubset(winners)) #create all subsets of size <=k
#     #S = set(s for s in powerset(C, k) if not set(s).issubset(winners)) #create all subsets of size <=k

#     for s in S:#iterate through all subsets
#         #l = len(s)
#         count = 0
#         for approval_list in approval_lists.values(): #iterate through all approval_lists/profile
#             if set(s).issubset(approval_list): #if s is a subset of the profile
#                 if abs(len(set(approval_list) & set(winners))) < l: #if profile intersection W is < l
#                     count += 1
#         if count >= l * n / k:
#             end_time = time.time()  # Record end time
#             duration = end_time - start_time  # Calculate duration
#             return "breaks EJR", duration
            
#     end_time = time.time()  # Record end time
#     duration = end_time - start_time  # Calculate duration
#     return "Satisfies EJR", duration

# W_range = range(1, 11)
# results = []

# # Iterate over different values of W
# for l in W_range:
#     for W_value in W_range:
#         W_set = create_set_W(greedy_list, W_value)
#         k = len(W_set)
#         non_winners = C - W_set

#         # Check for JR
#         result1, duration1 = JR(non_winners, approval_dict, W_set, k, n)

#         # Check for EJR
#         result2, duration2 = EJR(approval_dict, W_set, k, n, C, l)

#         results.append((W_value, result1, duration1, result2, duration2))

#         results_df = pd.DataFrame(results, columns=['W', 'JR', 'JR Duration', 'EJR', 'EJR Duration'])

#         print(tabulate(results_df, headers='keys', tablefmt='pretty'))


In [31]:

def powerset(iterable, max_size): 
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, min(len(s)+1, max_size+1)))
def subset_of_len_l(C_modified, l):
    return set(combinations(C_modified, l))

# Define the function to create the set W dynamically based on its size
def create_set_W(df, W):
    return set(df.head(W)['Value'])
# Define the function for JR
def JR(non_winners, approval_dict, W_set, k, n):
    start_time = time.time()  # Record start time
    counts = {}
    breaks_jr = False

    for non_winner in non_winners:
        counts[non_winner] = 0
        for key, value in approval_dict.items():
            if non_winner in value and not set(W_set).intersection(value):
                counts[non_winner] += 1
                if counts[non_winner] >= n / k:
                    breaks_jr = True
                    break
        if breaks_jr:
            return "Breaks JR."
        
    end_time = time.time()  # Record end time
    duration = end_time - start_time  # Calculate duration
    return "Satisfies JR.", duration

# Define the function for EJR
def EJR(approval_lists, winners, k, n, C,l):
    start_time = time.time()  # Record start time
    
    #FOR L in 1 to k+1, 
    #new C= where C have atleast nl/k votes 
    C_modified = [candidate for candidate in C if greedy_list.loc[greedy_list['Value'] == candidate, 'Count'].iloc[0] >= l * n / k]
    #print("lenght of C=",len(C))
    #print("lenght of C_modified=",len(C_modified))
    # Generate subsets S using C_modified
    
    # S should be only len =l
    S=subset_of_len_l(C_modified, l)
    #S = set(s for s in powerset(C_modified, k) if not set(s).issubset(winners)) #create all subsets of size <=k
    #S = set(s for s in powerset(C, k) if not set(s).issubset(winners)) #create all subsets of size <=k

    for s in S:#iterate through all subsets
        #l = len(s)
        count = 0
        for approval_list in approval_lists.values(): #iterate through all approval_lists/profile
            if set(s).issubset(approval_list): #if s is a subset of the profile
                if abs(len(set(approval_list) & set(winners))) < l: #if profile intersection W is < l
                    count += 1
        if count >= l * n / k:
            end_time = time.time()  # Record end time
            duration = end_time - start_time  # Calculate duration
            return "breaks EJR", duration
            
    end_time = time.time()  # Record end time
    duration = end_time - start_time  # Calculate duration
    return "Satisfies EJR", duration

W_range = range(1, 11)
results = []

# Iterate over different values of W
for l in W_range:
    for W_value in W_range:
        W_set = create_set_W(greedy_list, W_value)
        k = len(W_set)
        non_winners = C - W_set

        # Check for JR
        result1, duration1 = JR(non_winners, approval_dict, W_set, k, n)

        # Check for EJR
        result2, duration2 = EJR(approval_dict, W_set, k, n, C, l)

        results.append((l,W_value, result1, duration1, result2, duration2))

        results_df = pd.DataFrame(results, columns=['l','k', 'JR', 'JR Duration', 'EJR', 'EJR Duration'])

    print(tabulate(results_df, headers='keys', tablefmt='pretty'))


+---+---+----+---------------+-----------------------+---------------+----------------------+
|   | l | k  |      JR       |      JR Duration      |      EJR      |     EJR Duration     |
+---+---+----+---------------+-----------------------+---------------+----------------------+
| 0 | 1 | 1  | Satisfies JR. | 0.006695985794067383  | Satisfies EJR | 0.016202688217163086 |
| 1 | 1 | 2  | Satisfies JR. | 0.0036852359771728516 | Satisfies EJR | 0.017058372497558594 |
| 2 | 1 | 3  | Satisfies JR. | 0.0035505294799804688 | Satisfies EJR | 0.019230127334594727 |
| 3 | 1 | 4  | Satisfies JR. | 0.0034263134002685547 | Satisfies EJR | 0.010501384735107422 |
| 4 | 1 | 5  | Satisfies JR. |  0.01562786102294922  | Satisfies EJR | 0.025886058807373047 |
| 5 | 1 | 6  | Satisfies JR. | 0.0055086612701416016 | Satisfies EJR |  0.031341552734375   |
| 6 | 1 | 7  | Satisfies JR. |          0.0          | Satisfies EJR | 0.03278684616088867  |
| 7 | 1 | 8  | Satisfies JR. | 0.0050046443939208984 | Satis